# Vorbereitung

In [1]:
%env KERAS_BACKEND=theano
%env THEANO_FLAGS=floatX=float32,device=cpu

env: KERAS_BACKEND=theano
env: THEANO_FLAGS=floatX=float32,device=cpu


In [2]:
%load_ext autoreload
%autoreload 1
%aimport bb_behavior
%aimport bb_behavior.plot
%aimport bb_behavior.tracking
%aimport bb_behavior.tracking.pipeline

import bb_behavior
import bb_behavior.plot
import bb_behavior.tracking
import bb_behavior.tracking.pipeline

import pandas as pd
import time
import datetime

import os
import glob

Using Theano backend.


In [3]:
from tqdm import tqdm_notebook # progress bar

import math
import numpy as np
from bb_tracking.data.constants import DETKEY
#from bb_tracking.tracking import score_id_sim_v
from bb_tracking.tracking import distance_orientations_v, distance_positions_v

from bb_behavior.tracking.pipeline import detect_markers_in_video
from bb_behavior.tracking.pipeline import track_detections_dataframe
from bb_behavior.tracking.pipeline import display_tracking_results

# Run

In [4]:
from bb_behavior.tracking.pipeline import get_default_pipeline
default_pipeline = None
default_pipeline = get_default_pipeline(localizer_threshold="0.50")

/home/tscho/.local/lib/python3.6/site-packages/keras/backend/theano_backend.py:1032: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/tscho/.local/lib/python3.6/site-packages/keras/backend/theano_backend.py:1032: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/tscho/.local/lib/python3.6/site-packages/keras/backend/theano_backend.py:1032: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


In [5]:
# Hilfsfunktionen
def filename_to_datestring(filname):
    """
    filename can be path
    """
    return os.path.split(filname)[-1].split('.')[0].split('_')[1]

def datestring_to_filename(datestring, prefix="e00_"):
    return config["videos_dir"] + prefix + datestring + ".h264"

def string_to_timestamp(datestring):
    """ 
    params
        string: format 2018-08-19-01-08-13
    output
        unix timestamp (float)
    """
    
    return time.mktime(time.strptime(datestring, "%Y-%m-%d-%H-%M-%S"))

def timestamp_to_string(timestamp):
    return time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime(timestamp))

In [6]:
# get all videos between timestamp_in and timestamp_out
def get_videos_between(timestamp_in, timestamp_out, all_paths):
    """ returns all video between timestamp_in and timestamp out (inclusive) """
    
    mask = (all_paths['video'] >= timestamp_in) & (all_paths['video'] <= timestamp_out)
    return list(all_paths[mask]['video'])

def loadGTD(path):
    # read in the test data csv
    test_data = pd.read_csv(path)

    # convert the full filenames to string timestamps, and sort by timestamp_in
    test_data['video'] = test_data['video'].apply(lambda x: filename_to_datestring(x))
    test_data['video_start_time'] = test_data['timestamp_in'].apply(lambda x: filename_to_datestring(x))
    test_data['video_end_time'] = test_data['timestamp_out'].apply(lambda x: filename_to_datestring(x))
    test_data.sort_values(['video_start_time'], inplace=True)

    test_data.drop('timestamp_in', 1, inplace=True)
    test_data.drop('timestamp_out', 1, inplace=True)
    test_data.drop('video', 1, inplace=True)
    return test_data

In [7]:
tmp = loadGTD("bees_test.csv")
tmp

,bee_id,in_direction,out_direction,video_start_time,video_end_time
102,1970,left,left,2018-08-19-02-21-37,2018-08-19-02-22-40
0,855,right,left,2018-08-19-05-25-51,2018-08-19-05-25-51
1,220,left,right,2018-08-19-05-25-51,2018-08-19-05-25-51
24,8692,left,right,2018-08-19-06-35-41,2018-08-19-06-35-41
106,1191,right,left,2018-08-19-07-10-38,2018-08-19-07-10-38
107,1671,right,left,2018-08-19-07-10-38,2018-08-19-07-10-38
56,1788,right,left,2018-08-19-07-16-49,2018-08-19-07-16-49
55,749,left,right,2018-08-19-07-16-49,2018-08-19-07-16-49
43,3925,left,right,2018-08-19-07-47-52,2018-08-19-07-47-52
89,481,left,right,2018-08-19-08-21-08,2018-08-19-08-21-29


In [25]:
config = dict(tag_pixel_diameter=50,
              n_frames=None,
              confidence_filter_detections=0.08,
              confidence_filter_tracks=0.20,
              coordinate_scale=1.0,
              start_time=None,
              fps=10.0,
              cam_id=0,
              left_leaving_area = 0.3, # Prozente vom Bildschirmrand, zB. bei 1000px und 0.15 -> 0-150px
              right_leaving_area = 0.3,
              px_x_resolution_vid = 1944,
              videos_dir = "../videos/videos_tags/")


In [9]:
def detect_tracks(paths, save_to_csv = False):
    # Hier passiert das eigentliche Tracken und speichern der Ergebnisse:
    num_processed_videos = 0
    video_data = dict()
    frame_info = None
    detections = None
    bad_paths = []

    for path in tqdm_notebook(paths):
        start_time = config["start_time"]
        cam_id = config["cam_id"]
        try:
            num_processed_videos += 1

            frame_info, detections = detect_markers_in_video(path,
                                                          decoder_pipeline=default_pipeline,#pipeline=pipelines(),
                                                         tag_pixel_diameter=config["tag_pixel_diameter"],
                                                          n_frames=config["n_frames"],
                                                          fps=config["fps"],
                                                         progress="tqdm_notebook"
                                                )
            # Sonst würden keine Tracks erkannt werden -> Fehlermeldung
            if len(detections[detections['confidence']>=config["confidence_filter_detections"]]) == 0:
                continue
            tracks = track_detections_dataframe(detections,
                                                tracker="tracker.det_score_fun.frag_score_fun.dill",
                                                confidence_filter_detections=config["confidence_filter_detections"],
                                               confidence_filter_tracks=config["confidence_filter_tracks"],
                                                coordinate_scale=config["coordinate_scale"],
                                               )
            date_string = filename_to_datestring(path)
            tracks['video'] = date_string
            video_data[path] = (frame_info, detections, tracks)
        except ValueError as err: #tritt auf, wenn Video leer ist. In diesem Fall: überspringe video
            try:
                bad_paths.append(path)
                # wir arbeiten später nochmal mit paths, daher müssen das leere löschen, weil sonst
                # in video_data kein zugehöriger Value zu Key = file zu finden ist.
            except KeyError as err:
                continue
        except Exception as err:
            print(err)
            raise
        # only first vid: break

    for bad_path in bad_paths:
        paths.remove(bad_path)

    #video_data["file"][0] --> frame-info, [1] --> detections, [2] --> tracks
    for path in paths:
        display_tracking_results(path, video_data[path][0], video_data[path][1], video_data[path][2])

    tracks = [video_data[paths[x]][2] for x in range(len(paths))]
    tracks = pd.concat(tracks,ignore_index=True)
    tracks = tracks.drop(columns=["localizerSaliency", "beeID", "camID", "frameIdx"])

    if save_to_csv:
        with open("tracks.csv", "w") as f:
            tracks.to_csv(f)

    return tracks

## a: Create video path list from all videos in videos_dir

In [10]:
# Einen Iterable speichern, der alle Videos in einem Iterable zur Verfügung stellt
# Diesen Iterable können wir dann in der nächsten Zelle mit tqdm schön durchlaufen
base_directory = config["videos_dir"]
paths = [i for i in os.listdir(base_directory) if i.endswith(".h264")]
for i in range(len(paths)):
    paths[i] = base_directory + paths[i]

## b: Create video path list from test set

In [11]:
# read in the test data csv
test_data = loadGTD("bees_test.csv")

all_videos = []

all_paths = pd.DataFrame(glob.glob(os.path.join(config["videos_dir"], '*.h264')), columns=['video'])
all_paths['video'] = all_paths['video'].apply(lambda x: filename_to_datestring(x))
all_paths.sort_values(['video'])

# go through test_data and get all videos between timestamp_in and timestamp_out
for index, row in test_data.iterrows():
    all_videos += get_videos_between(row['video_start_time'],row['video_end_time'], all_paths)
    
all_videos = sorted(list(set(all_videos)))
paths = [datestring_to_filename(x) for x in all_videos]
del all_paths
del test_data

# Continue here

## Zwischenschritt: Merge all close Tracks of one bee

In [15]:
# RUN THIS IF TRACKS WHERE ALREADY CALCULATED AND SAVED TO AN .csv
# CSV EINLESEN UND SETZEN
tracks = pd.read_csv("all_tracks.csv", index_col=0)

In [ ]:
tracks

In [13]:
def gather_tracks(tracks):
    """
    transform the tracks df to : bee_id, [list of positions (x,y)], [list of timestamps], timestamp_of video
    """

    #  transform tracks df to : bee_id, [list of positions (x,y)], [list of timestamps], timestamp_of video
    grouped = tracks.groupby(['bee_id','track_id','video'],  as_index=False)['xpos','ypos','timestamp','zrotation']

    tracks_ml = grouped.aggregate(lambda x: list(x))

    # we don't need track_id anymore
    tracks_ml = tracks_ml.drop('track_id', 1)

    # add a column: convert video name to timestamp
    tracks_ml['video_start_time'] = tracks_ml['video'].apply(lambda x: string_to_timestamp(x))
    
    # we don't need video anymore
    tracks_ml = tracks_ml.drop('video', 1)

    # because aggregated: now multiple timestamps per row: --> rename
    tracks_ml = tracks_ml.rename(columns={'timestamp': 'timestamps'})

    # calculate start time of track by adding timestamp of track (seconds since start of video)
    # to timestamp of video (date)
    tracks_ml['track_start_time'] = tracks_ml['video_start_time'] + tracks_ml['timestamps'].apply(lambda x: x[0])
    tracks_ml['track_end_time'] = tracks_ml['video_start_time'] + tracks_ml['timestamps'].apply(lambda x: x[-1])
    
    # convert back to string
    tracks_ml['video_start_time'] = tracks_ml['video_start_time'].apply(lambda x: timestamp_to_string(x))
    
    all_paths = pd.DataFrame(glob.glob(os.path.join(config["videos_dir"], '*.h264')), columns=['video'])
    all_paths['video'] = all_paths['video'].apply(lambda x: filename_to_datestring(x))
    all_paths.sort_values(['video'])
    # get end video
    tracks_ml['video_end_time'] =  tracks_ml['track_end_time'].apply(lambda x: timestamp_to_string(x))
    tracks_ml['video_end_time'] =  tracks_ml[['video_start_time','video_end_time']].apply(lambda x: get_videos_between(x[0],x[1],all_paths)[-1], axis=1)

    return tracks_ml

def merge_tracks(tracks_ml, verbose = False):
    """
    in: 
        tracks_ml: output from gather_tracks(tracks)
    out:
        same df as in, with merged rows
        [bee_id:Float, xpos:[Float], ypos:[Float], zrotation:[Float], timestamps:[Float], video_start_time: String,
        video_end_time: String, track_start_time:Float, track_end_time:Float]
        
    merge tracks of same bee where start and end timestamps are close together
    assume there can not be overlapping tracks
    """
    
    
    # 1. sort: bee_id, start_time
    tracks_ml = tracks_ml.sort_values(['bee_id', 'track_start_time'])

    
    # first convert to timestamp
    tracks_ml['video_start_time'] = tracks_ml['video_start_time'].apply(lambda x: string_to_timestamp(x))
    
    index = 0
    while(True):
        row = tracks_ml.iloc[index]
        next_row = tracks_ml.iloc[index+1]

        # if the tracks are from same bee and the start time of next row is closer then 10s -> merge the rows
        # merge rows means, next_row is deleted
        if (row['bee_id'] == next_row['bee_id']) and ((next_row['track_start_time'] - row['track_end_time']) < 10):
            
            if verbose:
                print(timestamp_to_string(row['video_start_time']), row['bee_id'], next_row['track_start_time'] - row['track_end_time'])
                print(next_row['video_start_time'] - row['video_start_time'])
                print(timestamp_to_string(row['track_start_time']),timestamp_to_string(row['track_end_time']),timestamp_to_string(row['video_start_time']),row['video_end_time'])
                print(row['timestamps'])
                print(row.name)
                print(timestamp_to_string(next_row['track_start_time']))
                print('----------------------------------')
           
            # update the timestamps of nextrow
            t = next_row['video_start_time'] - row['video_start_time']
            timestamps = list( np.array(next_row['timestamps']) + t)

            # merge xpos, ypos, timestamps lists
            tracks_ml.at[row.name,'xpos'] = row['xpos']+next_row['xpos']
            tracks_ml.at[row.name,'ypos'] = row['ypos']+next_row['ypos']
            tracks_ml.at[row.name,'timestamps'] = row['timestamps']+timestamps

            # update end_time
            tracks_ml.at[row.name,'track_end_time'] = next_row['track_end_time']
            tracks_ml.at[row.name,'video_end_time'] = next_row['video_end_time']

            # delete the merged row (next_row)
            tracks_ml = tracks_ml.drop(next_row.name).copy()

        else:
            index += 1
        
        if index == len(tracks_ml) - 1:
            break
            
    # convert back to string
    tracks_ml['video_start_time'] = tracks_ml['video_start_time'].apply(lambda x: timestamp_to_string(x))
            
    return tracks_ml

In [59]:
tracks_ml = gather_tracks(tracks)
tracks_ml = merge_tracks(tracks_ml)
tracks_ml = tracks_ml.sort_values(['video_start_time'])

# Accuracy of Tracking

In [60]:
# read in the test data csv (the manually labeled data)
test_data = loadGTD("bees_test.csv")
test_data.drop_duplicates(inplace=True)

In [61]:
test_data_len = len(test_data)

In [63]:
# get the total number of tracks where the start_video and bee_id is correct
correct_in_video_start = len(pd.merge(tracks_ml,test_data[['bee_id','in_direction','video_start_time']], on=['bee_id','video_start_time'], how='inner'))

# get the total number of tracks where the end video and bee_id is correct
correct_in_video_end = len(pd.merge(tracks_ml,test_data[['bee_id','out_direction','video_end_time']], on=['bee_id','video_end_time'], how='inner'))

# get all tracks where bee_id, video_start_time and video_end_time are the same
total_correct = len(pd.merge(tracks_ml,test_data[['bee_id','in_direction','out_direction','video_end_time','video_start_time']], how='inner',on=['bee_id','video_start_time','video_end_time']))

print('bee_id and video_start_time correct: %.2f (%d/%d)' % (correct_in_video_start/test_data_len,correct_in_video_start,test_data_len))
print('bee_id and video_end_time correct: %.2f (%d/%d)' % (correct_in_video_end/test_data_len,correct_in_video_end,test_data_len))
print('bee_ids and both videos correct: %.2f (%d/%d)' % (total_correct/test_data_len,total_correct,test_data_len))

bee_id and video_start_time correct: 0.67 (70/104)
bee_id and video_end_time correct: 0.66 (69/104)
bee_ids and both videos correct: 0.60 (62/104)


# Add labels to tracks data

In [64]:
# get all tracks where bee_id, video_start_time and video_end_time are the same
tracks_ml = pd.merge(tracks_ml,test_data,how='inner',on=['bee_id','video_start_time','video_end_time'])

In [135]:
print(len(tracks_ml))
tracks_ml.head()

62


,bee_id,xpos,ypos,timestamps,zrotation,video_start_time,track_start_time,track_end_time,video_end_time,in_direction,out_direction
0,1970.0,"[1175.1580028533936, 1275.0042209625246, 1199....","[100.45368194580078, 252.97906684875488, 264.9...","[1.6, 2.3, 2.8, 3.1, 4.0, 4.1, 4.2, 4.3, 4.6, ...","[-1.5437196000000002, 0.18208171, 1.7509313000...",2018-08-19-02-21-37,1.534638e+09,1.534638e+09,2018-08-19-02-22-40,left,left
1,220.0,"[587.6349468231201, 577.2753982543945, 589.496...","[123.45260810852051, 125.9588794708252, 138.67...","[0.4, 14.0, 14.1, 14.2, 14.3, 14.4, 14.5, 14.6...","[-2.787043, 0.020939333, -0.15247042, 0.053603...",2018-08-19-05-25-51,1.534649e+09,1.534649e+09,2018-08-19-05-25-51,left,right
2,3925.0,"[538.5966873168945, 588.3075313568115, 651.012...","[126.37010765075684, 126.80798149108887, 127.0...","[11.9, 12.0, 12.1, 12.2, 12.3, 12.4, 12.5, 12.6]","[-0.48655950000000003, -0.23565856, -0.0598887...",2018-08-19-07-47-52,1.534658e+09,1.534658e+09,2018-08-19-07-47-52,left,right
3,388.0,"[1375.2675552368164, 1313.4858207702634, 1225....","[239.0948257446289, 238.7294406890869, 237.849...","[0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1]","[2.9390156000000003, 2.9522842999999996, -2.97...",2018-08-19-08-29-06,1.534660e+09,1.534660e+09,2018-08-19-08-29-06,right,left
4,1066.0,"[1413.0752410888672, 1350.3371124267578, 1325....","[251.15822410583496, 238.2443084716797, 188.21...","[1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, ...","[2.8809676, 3.0900168, -2.7678201000000002, -3...",2018-08-19-09-28-45,1.534664e+09,1.534664e+09,2018-08-19-09-28-45,right,left


In [ ]:
# split into training and test data
mask = np
tracks_ml_train = 

# Baseline-Algorithmen

## Algorithmus 1: Baseline - Areas as Decider

In [67]:
from math import pi
""" Vorgehen:
Für jedes Bienen-Track Paar
- Prüfe, wo sich die Biene beim ersten erkannten Erscheinen aufhält
    - Ordne die Position in left, right oder middle ein
- Prüfe, wo sich die Biene beim letzten erkannten Erscheinen aufhält
    - Ordne die Position in left, right oder middle ein
Wenn sich Biene in der Mitte befindet, starte neue Routine, die links oder rechts zuordnet
Daraus kann nun abgeleitet werden, wo die Biene reingekommen ist und wo sie rausgegangen ist.
"""
# param: tracks_ml, s. oben
def baseline_alg_classify_bee(tracks_ml):
    def get_direction(xpos, zpos):
        #Helpers
        def is_left(xpos):
            return xpos <= config["left_leaving_area"]*config["px_x_resolution_vid"]
        def is_right(xpos):
            return xpos >= config["px_x_resolution_vid"] - config["right_leaving_area"]*config["px_x_resolution_vid"]
        # Routine, wenn Biene in der Mitte
        # Betrachte zRotation, das ist die Richtung in die die Biene guckt (in Bogenmaß)
        # gebe diese Richtung aus
        def middle_classifier(zRotation):
            if abs(zRotation) > pi/2:
                return "left"
            else:
                return "right"
            
        # Eintrittsseite festlegen
        if is_left(xpos):
            return "left"
        elif is_right(xpos):
            return "right"
        else:
            return middle_classifier(zpos)
        
    pred_in_direction = len(tracks_ml) * [None]
    pred_out_direction = len(tracks_ml) * [None]
    i = 0
    for index, row in tracks_ml.iterrows():
        pred_in_direction[i] = get_direction(row['xpos'][0], row['zrotation'][0])
        pred_out_direction[i]= get_direction(row['xpos'][-1], row['zrotation'][-1])
        i += 1
    
    #remove those with no 
    return pd.DataFrame({"pred_in_direction":pred_in_direction, "pred_out_direction":pred_out_direction})

## Algorithmus 2: Baseline - zpos as decider

In [94]:
from math import pi
""" Vorgehen:
Unterschied: Hier wird nicht auf die Position abgestellt, sondern ausschließlich auf die
Richtung, in die die Biene schaut
"""
# param: tracks_ml, s. oben
def baseline_2_alg_classify_bee(tracks_ml):
    def get_in_direction(zpos):
        # Betrachte zRotation, das ist die Richtung in die die Biene guckt (in Bogenmaß)
        # gebe diese Richtung aus
        if abs(zpos) > abs(pi/2):
            return "right"
        else:
            return "left"
    def get_out_direction(zpos):
        # Betrachte zRotation, das ist die Richtung in die die Biene guckt (in Bogenmaß)
        # gebe diese Richtung aus
        if abs(zpos) > abs(pi/2):
            return "left"
        else:
            return "right"
        
    pred_in_direction = len(tracks_ml) * [None]
    pred_out_direction = len(tracks_ml) * [None]
    i = 0
    for index, row in tracks_ml.iterrows():
        pred_in_direction[i] = get_in_direction(row['zrotation'][0])
        pred_out_direction[i]= get_out_direction(row['zrotation'][-1])
        i += 1
    
    return pd.DataFrame({"pred_in_direction":pred_in_direction, "pred_out_direction":pred_out_direction})

In [ ]:
from math import pi
""" Vorgehen:
Unterschied: Hier wird nicht auf die Position abgestellt, sondern ausschließlich auf die
Richtung, in die die Biene schaut
"""
#results = {"bee_id":[], "time_in":[], "os_in":[], "time_out":[], "os_out":[], "in_direction":[], "out_direction":[]}
# param: tracks_ml, s. oben
def baseline_2_alg_classify_bee(tracks_ml):
    def get_in_direction(zpos):
        # Betrachte zRotation, das ist die Richtung in die die Biene guckt (in Bogenmaß)
        # gebe diese Richtung aus
        if zpos > abs(pi/2):
            return "right"
        else:
            return "left"
    def get_out_direction(zpos):
        # Betrachte zRotation, das ist die Richtung in die die Biene guckt (in Bogenmaß)
        # gebe diese Richtung aus
        if zpos > abs(pi/2):
            return "left"
        else:
            return "right"
    
    # "bee_id":[int], "time_in":[String], "os_in":[Float], "time_out":[String],
    # "os_out":[Float], "in_direction":[{"left", "right"}], "out_direction":[{"left", "right"}]
    results = {"bee_id":[], "time_in":[], "os_in":[], "time_out":[], "os_out":[], "in_direction":[],
               "out_direction":[]}
        
    for row in tracks_ml.itertuples(index=False):
        results["bee_id"].append(row[0])
        results["time_in"].append(row[5])
        results["os_in"].append(row[-3] - string_to_timestamp(row[5])) #track_start_time - video_start_time
        results["time_out"].append(row[-1])
        results["os_out"].append(row[-2] - string_to_timestamp(row[-1])) #track_end_time - video_end_time
        results["in_direction"].append(get_in_direction(row[4][0]))
        results["out_direction"].append(get_out_direction(row[4][-1]))
    
    return results

In [132]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

def conbine_in_out(y_in, y_out):
    """ convert in_direction, out_direction to ll, lr, rr, rl """
    
    in_label = y_in.copy()
    in_label[y_in == 'left'] = 'L'
    in_label[y_in == 'right'] = 'R'
    
    out_label = y_out.copy()
    out_label[y_out == 'left'] = 'L'
    out_label[y_out == 'right'] = 'R'

    return in_label + out_label
    

def print_classifier_metrics(y, ypred):
    
        print('Confusion Matrix In-Direction:')
        print(pd.DataFrame(confusion_matrix(tracks_ml['in_direction'], ypred['pred_in_direction']), columns=['L','R'], index=['L','R']))
        print('Accuracy In-Direction:',accuracy_score(tracks_ml['in_direction'], ypred['pred_in_direction']))
        print('\n')
        print('Confusion Matrix Out-Direction:')
        print(pd.DataFrame(confusion_matrix(tracks_ml['out_direction'], ypred['pred_out_direction']), columns=['L','R'], index=['L','R']))
        print('Accuracy Out-Direction:',accuracy_score(tracks_ml['out_direction'], ypred['pred_out_direction']))
        
        y_combined = conbine_in_out(y['in_direction'], y['out_direction'])
        y_pred_combined = conbine_in_out(ypred['pred_in_direction'], ypred['pred_out_direction'])
        print('\n')
        print('Confusion Matrix Combined:')
        print(pd.DataFrame(confusion_matrix(y_combined,y_pred_combined, labels=['LL', 'LR', 'RL', 'RR']), columns=['LL', 'LR', 'RL', 'RR'], index=['LL', 'LR', 'RL', 'RR']))
        print('Total Accuracy:', accuracy_score(y_combined, y_pred_combined))

In [133]:
results = baseline_alg_classify_bee(tracks_ml)
print_classifier_metrics(tracks_ml[['in_direction','out_direction']], results[['pred_in_direction','pred_out_direction']])

Confusion Matrix In-Direction:
    L   R
L  33   7
R   8  14
Accuracy In-Direction: 0.7580645161290323


Confusion Matrix Out-Direction:
    L   R
L  29   1
R   5  27
Accuracy Out-Direction: 0.9032258064516129


Confusion Matrix Combined:
    LL  LR  RL  RR
LL  12   1   1   0
LR   3  17   1   5
RL   4   0  12   0
RR   1   3   0   2
Total Accuracy: 0.6935483870967742


In [134]:
results = baseline_2_alg_classify_bee(tracks_ml)
print_classifier_metrics(tracks_ml[['in_direction','out_direction']], results[['pred_in_direction','pred_out_direction']])

Confusion Matrix In-Direction:
    L   R
L  37   3
R   3  19
Accuracy In-Direction: 0.9032258064516129


Confusion Matrix Out-Direction:
    L   R
L  25   5
R   5  27
Accuracy Out-Direction: 0.8387096774193549


Confusion Matrix Combined:
    LL  LR  RL  RR
LL  10   3   1   0
LR   3  21   0   2
RL   0   2  14   0
RR   0   1   2   3
Total Accuracy: 0.7741935483870968


# Algorithmus 3: Machine Learning

## Vorbereiten Lernvideos, Prüfvideos

### Optional: Tracks um 180° drehen; Tracks horizontal invertieren; Tracks vertikal invertieren (Funktioniert, abhängig von der Wichtigkeit der Zeit als Feature, vielleicht nicht!)

In [ ]:
# Tracks um 180° drehen
track_rotated = tracks_ml


## 3.1 Lernen Logistische Regression

## 3.1 Alg: Logistische Regression

## 3.2 Lernen Decision Tree

## 3.2 Alg Decision Tree

# Vergleichsfunktion Algorithmus mit Labels

In [ ]:
def getAccuracy():
    """ 
    Input: bees_test.csv, MachineLearnedData.csv
    Output: "result" as DataFrame with an overview of compareable bees adn their correct/wrong correctness
    
    Read in two csv file and store values in DataFrame's. Merge DataFrame's to get exact correctness of 
    total-, in- and out_direction of bees regarding their time_in and time_out relation. 
    """

    # delete existing result.csv file to prevent errors
    if (os.path.isfile('./result.csv')):
        os.remove("result.csv")

    # store data in dataFrame
    gtd = loadGTD('bees_test.csv')
    mld = pd.read_csv('MachineLearnedData.csv')

    # change column name to merge dataFrames
    gtd.rename(columns = {'video_start_time':'time_in', 'video_end_time':'time_out'}, inplace=True)

    # delete dublications
    mld = mld.drop_duplicates(['bee_id', 'time_in', 'time_out'], keep='first')
    gtd = gtd.drop_duplicates(['bee_id', 'time_in', 'time_out'], keep='first')

    # change DT from float to int to have the same DT like gtd
    mld.bee_id = mld.bee_id.astype('int32')

    # merge to get 100% correct matches
    mergeTotalCorrect = pd.merge(gtd, mld, on=['bee_id', 'time_in', 'time_out', 'in_direction', 'out_direction'], how='inner')
    #merge to get in_direction correct
    mergeInDirection = pd.merge(gtd, mld, on=['bee_id', 'time_in', 'in_direction'], how='inner')
    #merge to get out_direction correct
    mergeOutDirection = pd.merge(gtd, mld, on=['bee_id', 'time_out', 'out_direction'], how='inner')
    # merge to get comparable bees to get all found bees
    mergeableBees = pd.merge(gtd, mld, on=['bee_id', 'time_in', 'time_out'], how='inner')

    # iterate mergeableBees for visualization
    result = pd.DataFrame()
    result['bee_id'] = mergeableBees['bee_id'] 
    result['in_equals'] = mergeableBees['in_direction_x'] == mergeableBees['in_direction_y']
    result['out_equals'] = mergeableBees['out_direction_x'] == mergeableBees['out_direction_y']
    #result['movement'] = result['in_equals'].bool() == True and result['out_equals'].bool() == False

    # store to result.csv
    result.to_csv('result.csv', encoding='utf-8', index=False)

    # calculate statitics
    inCorrect = round((result['in_equals'].sum()/len(result) * 100), 2)
    outCorrect = round((result['out_equals'].sum()/len(result) * 100), 2)
    allCorrect = round(((result['in_equals'] == result['out_equals']).sum()/len(result) * 100), 2)
    aveCorrect = round(((allCorrect + inCorrect + outCorrect) / 3), 2)

    # print statistics
    try:
        print(len(gtd), 'entries in GroundTruthData.', len(mergeableBees), 'entries compareable in MachineLearnedData: ', round((len(mergeableBees)/len(gtd))*100, 2), '%' )
        print(allCorrect, "% total correctness.")
        print(inCorrect, "% Ingoing correctness.")
        print(outCorrect, "% Outgoing correctness.")
        print(aveCorrect, "% Average correctness.")
        print("Accuracy of ",round((len(mergeableBees)/len(gtd))*100*(allCorrect/100),2),"%.")
    except ZeroDivisionError:
        print("Nothing to compare!")

    # end compare algorithm
    print('\nCheck result.csv file or hit \'result\' for more details.')
    print('Use DataFrames for deeper information: \n\'mergeTotalCorrect\',\'mergeInCorrect\',\'mergeOutCorrect\',\'mergeableBees\' ')


In [ ]:
getAccuracy()